In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib as plt
import numpy as np

In [2]:
business = pd.read_json('yelp_dataset/business.json', lines=True)
#checkin = pd.read_json('yelp_dataset/checkin.json', lines=True)
#photo = pd.read_json('yelp_dataset/photo.json', lines=True)
##review = pd.read_json('yelp_dataset/review.json', lines=True)
#tip = pd.read_json('yelp_dataset/tip.json', lines=True)
##user = pd.read_json('yelp_dataset/user.json', lines=True)

In [3]:
lv_business = business[business.city=="Las Vegas"]
print(lv_business.shape)
lv_business.head()

(29370, 14)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
7,"4545 E Tropicana Rd Ste 8, Tropicana","{'RestaurantsPriceRange2': '3', 'GoodForKids':...",gbQN7vr_caG_A1ugSmGhWg,"Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,36.099872,-115.074574,Supercuts,89121,3,3.5,NV
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,None,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV
18,,{'BusinessAcceptsCreditCards': 'True'},nh_kQ16QAoXWwqZ05MPfBQ,"Event Planning & Services, Photographers, Prof...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.116549,-115.088115,Myron Hensel Photography,89121,21,5.0,NV
20,"450 Fremont St, Ste 179",{'GoodForKids': 'True'},dFMxzHygTy6F873843dHAA,"Arcades, Arts & Entertainment",Las Vegas,"{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ...",1,36.169993,-115.140685,Fremont Arcade,89101,38,4.5,NV
22,6007 Dean Martin Dr,"{'BikeParking': 'False', 'Ambience': '{'romant...",KWywu2tTEPWmR9JnBc0WyQ,"Nightlife, Arts & Entertainment, Bars, Strip C...",Las Vegas,"{'Thursday': '19:30-2:0', 'Friday': '19:30-3:0...",1,36.080168,-115.182756,Hunk Mansion,89118,107,4.0,NV


In [4]:
#skipping attributes for now...
#business_x = business[['attributes', 'categories']]
#business_y = business[['stars']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(lv_business['categories'], \
                                                    lv_business['stars'], test_size=0.33, random_state=42)


#### need to convert 'categories' or 'attributes' to categorical data

#### One-hot Encoding for Categorical Variables

X_train = pd.concat([X_train, pd.get_dummies(X_train['workclass'])], axis=1)

X_test = pd.concat([X_test, pd.get_dummies(X_test['workclass'])], axis=1)

X_train = X_train.drop('workclass', axis=1)

X_test = X_test.drop('workclass', axis=1)

In [6]:
X_train = pd.Series(X_train).str.get_dummies(sep=',')
X_test = pd.Series(X_test).str.get_dummies(sep=',')

In [7]:
#check for columns missing from test set
missing_train = list(set(list(X_train)) - set(list(X_test)))
for col in missing_train:
    X_test[col] = 0
    
missing_test = list(set(list(X_test)) - set(list(X_train)))
for col in missing_test:
    X_train[col] = 0

In [8]:
#X_train = X_train.drop('categories', axis=1)
#X_test = X_test.drop('categories', axis=1)

In [9]:
X_train.head()

,& Probates,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,Active Life,Acupuncture,Addiction Medicine,Adoption Services,...,Immunodermatologists,Honduran,Memory Care,Bike Shop,Popcorn Shops,Drones,Sailing,Nicaraguan,Home Developers,Art Restoration
120012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99635,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127851,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X_test.head()

,& Probates,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,Active Life,Acupuncture,Addiction Medicine,Adult,...,Knife Sharpening,Yelp Events,Aircraft Repairs,Musical Instrument Services,Tasting Classes,Public Art,Livestock Feed & Supply,Adoption Services,Community Gardens,Kickboxing
185692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(19677, 2089)
(9693, 2089)
(19677,)
(9693,)


In [ ]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)
print(clf.coef_)
print(clf.intercept_) 

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.score(y_pred, y_test) 

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
le.classes_
le.transform(y_train) 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, le.transform(y_train)) 

In [ ]:
y_pred = neigh.predict(X_test)

In [ ]:
y_pred.reshape(-1,1)

In [ ]:
le.transform(y_test).reshape(-1,1)

In [ ]:
le_test = preprocessing.LabelEncoder()
le_test.fit(y_test)

neigh.score(y_pred.reshape(-1,1), le_test.transform(y_test).reshape(-1,1)) 

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
y_pred_arr = y_pred.reshape(1,-1)

In [ ]:
pd.Series(y_pred)

In [ ]:
le.transform(y_test)

In [ ]:
le_test = preprocessing.LabelEncoder()
le_test.fit(y_test)

neigh.score(pd.Series(y_pred).reshape(-1,1), pd.Series(le.transform(y_test).reshape(-1,1)) )